# Capstone
#### This is the working notebook for my capstone project

In [85]:
import pandas as pd
import numpy as np


In [86]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Week 3: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd           #Import Pandas for dataframe
import requests               #Import requests to get source code from wiki
from bs4 import BeautifulSoup #Import BeautifulSoup to scrape table from source code

### Scraping Data From Wiki Into Dataframe:

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Drop Non-assigned Boroughs:

In [3]:
#df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
#df=df.reset_index(drop=True)
df=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### For Non-assigned  Neighbourhoods, Rename Neighbourhood to Borough Name:

In [4]:
dfna=df[df['Neighbourhood']=='Not assigned']
dfna['Neighbourhood']=dfna['Borough']
df=pd.merge(df,dfna, on='Borough',how='left')

df.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Postcode_x,Borough,Neighbourhood_x,Postcode_y,Neighbourhood_y
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,NaN,NaN
3,M5A,Downtown Toronto,Regent Park,NaN,NaN
4,M6A,North York,Lawrence Heights,NaN,NaN
5,M6A,North York,Lawrence Manor,NaN,NaN
6,M7A,Queen's Park,Not assigned,M7A,Queen's Park
7,M9A,Etobicoke,Islington Avenue,NaN,NaN
8,M1B,Scarborough,Rouge,NaN,NaN
9,M1B,Scarborough,Malvern,NaN,NaN


In [5]:
df.loc[df['Neighbourhood_x'] == 'Not assigned', ['Neighbourhood_x']] = df['Neighbourhood_y']
df=df.drop(['Neighbourhood_y', 'Postcode_y'], axis=1)
df=df.rename(columns={"Postcode_x": "PostalCode", "Neighbourhood_x": "Neighbourhood"})
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Group Neighbourhoods That Share The Same PostalCode and Display As Comma Separated List:

In [6]:
df1 = df.groupby(['PostalCode','Borough'],sort=False).agg(lambda x: ', '.join(x)).reset_index()

In [13]:
df1.head(15)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [14]:
df1.shape

(103, 3)